In [1]:
import random
import pandas as pd
from datetime import datetime

# Temporary workflow
# (1) Download csv with `heroku run make exportdb && make importdb`
# (2) save links to ~/Documents/links.txt in onetab format
# (3) run this script (or otherwise modify the CSV as desired)
# (4) check date cell
# (5) Upload back up with `make csv_to_s3 && heroku run make importdb`
# (6) run `make pipeline` to clean DB
links = pd.read_csv('data/export.csv')
links

/var/folders/3q/_qq6n78n6m92_765npwz342r0000gn/T/ipykernel_80120/564014651.py:12: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  links = pd.read_csv('data/export.csv')


,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1.0,ethics,custom,17.0,0.0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1.0,giving,custom,59.0,0.0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1.0,productivity,custom,56.0,0.0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1.0,politicalscience,custom,29.0,0.0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1.0,technology,custom,92.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
103030,165926,https://www.askamanager.org/2023/09/its-your-f...,it’s your Friday good news,\N,\N,2023-09-02 12:23:00.042734+00,2023-09-02 12:23:00.042734+00,NaN,\N,AskManager,12.0,0.0,\N
103031,165927,https://www.slowboring.com/p/friday-thread-9e0,Friday Thread,\N,\N,2023-09-02 12:23:00.04398+00,2023-09-02 12:23:00.04398+00,NaN,\N,Yglesias,45.0,0.0,\N
103032,165928,https://www.centreforeffectivealtruism.org/blo...,CEA’s 2017 review and 2018 plans,\N,www.centreforeffectivealtruism.org,2023-09-02 12:23:00.045222+00,2023-09-02 12:24:31.417088+00,0.0,\N,CEA,96.0,0.0,\N
103033,165929,https://www.askamanager.org/2023/09/weekend-op...,"weekend open thread – September 2-3\, 2023",\N,\N,2023-09-02 12:23:00.046426+00,2023-09-02 12:23:00.046426+00,NaN,\N,AskManager,29.0,0.0,\N


In [2]:
links['added'].max()

'2023-09-02 12:23:00.047626+00'

In [3]:
links[(links['aggregator'] == 'Custom') & (links['liked'] != '1') & (links['liked'] != '0') & (links['liked'] != '-1')]['id']

1299       1306
1300       1307
1301       1308
1302       1309
1303       1310
          ...  
99235    161931
99236    161933
99237    161934
99238    161935
99239    161936
Name: id, Length: 11962, dtype: int64

In [4]:
links[links['url'].duplicated()]

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
103032,165928,https://www.centreforeffectivealtruism.org/blo...,CEA’s 2017 review and 2018 plans,\N,www.centreforeffectivealtruism.org,2023-09-02 12:23:00.045222+00,2023-09-02 12:24:31.417088+00,0.0,\N,CEA,96.0,0.0,\N


In [5]:
links[links['url'].duplicated()]['id']

103032    165928
Name: id, dtype: int64

In [6]:
link_file = open('/Users/peterhurford/Documents/links.txt', 'r')
new_links = link_file.readlines()

In [7]:
def new_entry(links, url, title):
    return {'id': links['id'].max() + 1,
            'url': str(url).strip().replace(',', ''),
            'title': str(title).strip().replace(',', ''),
            'summary': '\\N',
            'domain': '\\N',
            'added': str(datetime.now()),
            'modified': str(datetime.now()),
            'liked': '\\N',
            'category': '\\N',
            'aggregator': 'Custom',
            'seed': random.randint(1, 100),
            'starred': 0,
            'tweet': 0}

for i, new_link in enumerate(new_links):
    if i % 10 == 0 or i == len(new_links) - 1:
        print('...{}/{}'.format(i + 1, len(new_links)))
    if ' | ' in new_link:
        new_link = new_link.replace('    - ', '').replace('\n', '')
        entry = new_entry(links, new_link.split(' | ')[0], new_link.split(' | ')[1])
        links = pd.concat([links, pd.DataFrame([entry])], ignore_index=True)

links

...1/1281
...11/1281
...21/1281
...31/1281
...41/1281
...51/1281
...61/1281
...71/1281
...81/1281
...91/1281
...101/1281
...111/1281
...121/1281
...131/1281
...141/1281
...151/1281
...161/1281
...171/1281
...181/1281
...191/1281
...201/1281
...211/1281
...221/1281
...231/1281
...241/1281
...251/1281
...261/1281
...271/1281
...281/1281
...291/1281
...301/1281
...311/1281
...321/1281
...331/1281
...341/1281
...351/1281
...361/1281
...371/1281
...381/1281
...391/1281
...401/1281
...411/1281
...421/1281
...431/1281
...441/1281
...451/1281
...461/1281
...471/1281
...481/1281
...491/1281
...501/1281
...511/1281
...521/1281
...531/1281
...541/1281
...551/1281
...561/1281
...571/1281
...581/1281
...591/1281
...601/1281
...611/1281
...621/1281
...631/1281
...641/1281
...651/1281
...661/1281
...671/1281
...681/1281
...691/1281
...701/1281
...711/1281
...721/1281
...731/1281
...741/1281
...751/1281
...761/1281
...771/1281
...781/1281
...791/1281
...801/1281
...811/1281
...821/1281
...831/1281
...

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1.0,ethics,custom,17.0,0.0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1.0,giving,custom,59.0,0.0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1.0,productivity,custom,56.0,0.0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1.0,politicalscience,custom,29.0,0.0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1.0,technology,custom,92.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
104311,167207,https://www.lesswrong.com/posts/fJqP9WcnHXBRBe...,Meta Questions about Metaphilosophy,\N,\N,2023-09-02 17:57:28.987001,2023-09-02 17:57:28.987006,\N,\N,Custom,72.0,0.0,NaN
104312,167208,https://joecarlsmith.com/2020/11/16/thoughts-o...,Thoughts on personal identity - Joe Carlsmith,\N,\N,2023-09-02 17:57:28.992009,2023-09-02 17:57:28.992014,\N,\N,Custom,14.0,0.0,NaN
104313,167209,https://joecarlsmith.com/2021/12/21/morality-a...,Morality and constrained maximization part 1 -...,\N,\N,2023-09-02 17:57:28.997030,2023-09-02 17:57:28.997036,\N,\N,Custom,6.0,0.0,NaN
104314,167210,https://drorpoleg.com/remote-is-not-enough/,Remote Is Not Enough,\N,\N,2023-09-02 17:57:29.002169,2023-09-02 17:57:29.002174,\N,\N,Custom,45.0,0.0,NaN


In [8]:
links[(links['aggregator'] == 'Custom') & (links['liked'] != '1') & (links['liked'] != '0') & (links['liked'] != '-1')]

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
1299,1306,http://blog.beeminder.com/trust/,Trusting Your Divided Self | Beeminder Blog,\N,blog.beeminder.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,productivity,Custom,10.0,0.0,0
1300,1307,http://powlyglot.com/fi3m-review-part-1-focus/,"""What I Learned from Fluent in 3 Months\, Part...",\N,powlyglot.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,productivity,Custom,9.0,0.0,0
1301,1308,http://effective-altruism.com/ea/u8/ea_intervi...,"""EA Interview Series\, February 2016: John Sal...",\N,effective-altruism.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,effectivealtruism,Custom,85.0,0.0,0
1302,1309,https://fivethirtyeight.com/features/most-2020...,Most 2020 Candidates Have Something In Common:...,\N,fivethirtyeight.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,1.0,politicalscience,Custom,82.0,0.0,0
1303,1310,http://www.smartpassiveincome.com/backlinking-...,THE Backlinking Strategy That Works – 2014 and...,\N,www.smartpassiveincome.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,entrepreneurship,Custom,59.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
104311,167207,https://www.lesswrong.com/posts/fJqP9WcnHXBRBe...,Meta Questions about Metaphilosophy,\N,\N,2023-09-02 17:57:28.987001,2023-09-02 17:57:28.987006,\N,\N,Custom,72.0,0.0,NaN
104312,167208,https://joecarlsmith.com/2020/11/16/thoughts-o...,Thoughts on personal identity - Joe Carlsmith,\N,\N,2023-09-02 17:57:28.992009,2023-09-02 17:57:28.992014,\N,\N,Custom,14.0,0.0,NaN
104313,167209,https://joecarlsmith.com/2021/12/21/morality-a...,Morality and constrained maximization part 1 -...,\N,\N,2023-09-02 17:57:28.997030,2023-09-02 17:57:28.997036,\N,\N,Custom,6.0,0.0,NaN
104314,167210,https://drorpoleg.com/remote-is-not-enough/,Remote Is Not Enough,\N,\N,2023-09-02 17:57:29.002169,2023-09-02 17:57:29.002174,\N,\N,Custom,45.0,0.0,NaN


In [11]:
def clean(txt):
    if not isinstance(txt, str):
        return txt
    txt = txt.replace('&nbsp;', '')
    txt = txt.replace('\\', '')
    txt = txt.replace('&ldquo;', '"')
    txt = txt.replace('&rdquo;', '"')
    txt = txt.replace('&lsquo;', '\'')
    txt = txt.replace('&rsquo;', '\'')
    txt = txt.replace('&mdash;', '-')
    txt = txt.replace('&ndash;', '-')
    for i in range(10):
        txt = txt.replace('"""', '"')
        txt = txt.replace('\'\'\'', '\'')
    txt = txt.replace('n"t', 'n\'t')
    txt = txt.replace('n""t', 'n\'t')
    return txt

links = links[links['id'].notnull()]   # Drop empty column
links['id'] = links['id'].astype(int)  # Fix float ID issue
links['seed'] = links['seed'].astype(int)
links['tweet'] = links['tweet'].apply(lambda x: 0 if str(x) == '\\N' else str(x).split('.')[0]).astype(int)
links['summary'] = links['summary'].apply(clean)
links['title'] = links['title'].apply(clean)
links['starred'] = links['starred'].fillna(0).apply(lambda x: 0 if str(x) == '\\N' else str(x).split('.')[0]).astype(int) # Backwards compatibility
links = links.sort_values('id')
links

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1.0,ethics,custom,17,0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1.0,giving,custom,59,0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1.0,productivity,custom,56,0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1.0,politicalscience,custom,29,0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1.0,technology,custom,92,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
104311,167207,https://www.lesswrong.com/posts/fJqP9WcnHXBRBe...,Meta Questions about Metaphilosophy,\N,\N,2023-09-02 17:57:28.987001,2023-09-02 17:57:28.987006,\N,\N,Custom,72,0,0
104312,167208,https://joecarlsmith.com/2020/11/16/thoughts-o...,Thoughts on personal identity - Joe Carlsmith,\N,\N,2023-09-02 17:57:28.992009,2023-09-02 17:57:28.992014,\N,\N,Custom,14,0,0
104313,167209,https://joecarlsmith.com/2021/12/21/morality-a...,Morality and constrained maximization part 1 -...,\N,\N,2023-09-02 17:57:28.997030,2023-09-02 17:57:28.997036,\N,\N,Custom,6,0,0
104314,167210,https://drorpoleg.com/remote-is-not-enough/,Remote Is Not Enough,\N,\N,2023-09-02 17:57:29.002169,2023-09-02 17:57:29.002174,\N,\N,Custom,45,0,0


In [17]:
links['modified'].tail(20)

104296    2023-09-02 17:57:28.911540
104297    2023-09-02 17:57:28.916554
104298    2023-09-02 17:57:28.921603
104299    2023-09-02 17:57:28.926652
104300    2023-09-02 17:57:28.931635
104301    2023-09-02 17:57:28.936670
104302    2023-09-02 17:57:28.941671
104303    2023-09-02 17:57:28.946778
104304    2023-09-02 17:57:28.951803
104305    2023-09-02 17:57:28.956844
104306    2023-09-02 17:57:28.961898
104307    2023-09-02 17:57:28.966924
104308    2023-09-02 17:57:28.971918
104309    2023-09-02 17:57:28.976956
104310    2023-09-02 17:57:28.981976
104311    2023-09-02 17:57:28.987006
104312    2023-09-02 17:57:28.992014
104313    2023-09-02 17:57:28.997036
104314    2023-09-02 17:57:29.002174
104315    2023-09-02 17:57:29.007360
Name: modified, dtype: object

In [12]:
links.to_csv('data/export.csv', index=False)